In [ ]:
# pip install pdf2image

In [3]:
from pdf2image import convert_from_path

# Store Pdf with convert_from_path function
pages = convert_from_path('Module0_Notes_Generator/pdfs/notes1.pdf',500)

for count, page in enumerate(pages):
    # Save pages as images in the pdf
    page.save('Module0_Notes_Generator/images/page'+ str(count) +'.jpg', 'JPEG')

In [6]:
import os
import base64
from groq import Groq

# Initialize the Groq client
client = Groq(
    api_key="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"
)

# Function to encode the image as base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Function to process images and save responses to a file
def process_images_and_save(folder_path, output_file):
    combined_text = ""
    
    for image_name in os.listdir(folder_path):
        if image_name.lower().endswith((".jpg", ".jpeg", ".png")):  # Check valid image extensions
            image_path = os.path.join(folder_path, image_name)
            base64_image = encode_image(image_path)
            
            try:
                # Send image query to Groq client
                chat_completion = client.chat.completions.create(
                    messages=[
                        {
                            "role": "user",
                            "content": [
                                {"type": "text", "text": "Extract the text from the given Image."},
                                {
                                    "type": "image_url",
                                    "image_url": {
                                        "url": f"data:image/jpeg;base64,{base64_image}",
                                    },
                                },
                            ],
                        }
                    ],
                    model="llama-3.2-11b-vision-preview",
                )
                
                # Append the response to the combined text
                response = chat_completion.choices[0].message.content
                combined_text += f"Response for {image_name}:\n{response}\n\n{'='*50}\n\n"
                
            except Exception as e:
                print(f"Error processing {image_name}: {e}")
    
    # Write the combined text to the output file
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(combined_text)
    print(f"Responses saved to {output_file}")

# Path to the folder containing images and output file
image_folder = "Module0_Notes_Generator/images"
output_file = "Module1_Notes_to_Quiz/notes.txt"

# Process images and save responses
process_images_and_save(image_folder, output_file)


Responses saved to Module1_Notes_to_Quiz/notes.txt
